In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [15]:
pip install evaluate

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Note: you may need to restart the kernel to use updated packages.


In [16]:
pip install peft==0.12.0

Note: you may need to restart the kernel to use updated packages.


导入所需的库

In [17]:
# 导入所需的python库
import os
import sys
import logging
import datasets
import evaluate

# 导入数据处理库
import pandas as pd
import numpy as np

# 从transformers库导入模型和分词器
from transformers import AutoModelForSequenceClassification, DebertaV2Tokenizer, DataCollatorWithPadding

# 导入训练器和训练参数
from transformers import Trainer, TrainingArguments

# 从peft库导入配置和模型获取函数
from peft import PromptTuningConfig, get_peft_model, TaskType

# 从sklearn库导入数据集分割函数
from sklearn.model_selection import train_test_split

In [18]:
! unzip /kaggle/input/word2vec-nlp-tutorial/labeledTrainData.tsv.zip
! unzip /kaggle/input/word2vec-nlp-tutorial/testData.tsv.zip
! unzip /kaggle/input/word2vec-nlp-tutorial/unlabeledTrainData.tsv.zip

Archive:  /kaggle/input/word2vec-nlp-tutorial/labeledTrainData.tsv.zip
replace labeledTrainData.tsv? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C
Archive:  /kaggle/input/word2vec-nlp-tutorial/testData.tsv.zip
replace testData.tsv? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C
Archive:  /kaggle/input/word2vec-nlp-tutorial/unlabeledTrainData.tsv.zip
replace unlabeledTrainData.tsv? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


读取训练和测试数据集

In [19]:
train = pd.read_csv("/kaggle/working/labeledTrainData.tsv", header=0, delimiter="\t", quoting=3)
test = pd.read_csv("/kaggle/working/testData.tsv", header=0, delimiter="\t", quoting=3)

设置日志记录的基本配置，并记录程序运行的开始

In [20]:
program = os.path.basename(sys.argv[0])
logger = logging.getLogger(program)

logging.basicConfig(format='%(asctime)s: %(levelname)s: %(message)s')
logging.root.setLevel(level=logging.INFO)
logger.info(r"running %s" % ''.join(sys.argv))

将训练数据集分割为训练集和验证集，其中20%作为验证集

In [21]:
train, val = train_test_split(train, test_size=.2)

创建字典，将标签和文本分别存储，以适配模型的输入格式。
并将字典转换为datasets库的数据集对象。

In [22]:
    train_dict = {'label': train["sentiment"], 'text': train['review']}
    val_dict = {'label': val["sentiment"], 'text': val['review']}
    test_dict = {"text": test['review']}

    train_dataset = datasets.Dataset.from_dict(train_dict)
    val_dataset = datasets.Dataset.from_dict(val_dict)
    test_dataset = datasets.Dataset.from_dict(test_dict)

指定了预训练模型的ID，并从指定的模型ID加载分词器。

In [23]:
    model_id = "microsoft/deberta-v3-large"

    tokenizer = DebertaV2Tokenizer.from_pretrained(model_id)

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


定义预处理函数，对文本进行分词处理，设置截断和填充。

In [24]:
def preprocess_function(examples):
        return tokenizer(examples['text'], truncation=True,padding='max_length', max_length=510)

对训练集、验证集和测试集应用预处理函数对文本进行分词处理，设置截断和填充。

In [25]:
tokenized_train = train_dataset.map(preprocess_function, batched=True)
tokenized_val = val_dataset.map(preprocess_function, batched=True)
tokenized_test = test_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

创建一个数据填充器，用于在训练时自动填充批次。并从指定的模型ID加载预训练模型。

In [26]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

model = AutoModelForSequenceClassification.from_pretrained(model_id)

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-large and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


定义LoRA配置并准备模型。

In [27]:
# Define LoRA Config
peft_config = PromptTuningConfig(
    num_virtual_tokens=10,
    task_type=TaskType.SEQ_CLS
)

# prepare int-8 model for training
#model = prepare_model_for_kbit_training(model)
# add LoRA adaptor
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 12,290 || all params: 435,076,100 || trainable%: 0.0028


加载准确率作为评估指标，并定义计算评估指标的函数，计算模型预测的准确率。

In [28]:
metric = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

定义了训练参数，包括输出目录、训练轮数、批次大小等。

In [29]:
training_args = TrainingArguments(
    output_dir='./checkpoint',  # output directory
    num_train_epochs=3,  # total number of training epochs
    per_device_train_batch_size=2,  # batch size per device during training
    per_device_eval_batch_size=4,  # batch size for evaluation
    warmup_steps=500,  # number of warmup steps for learning rate scheduler
    weight_decay=0.01,  # strength of weight decay
    logging_dir='./logs',  # directory for storing logs
    logging_steps=100,
    save_strategy="no",
    evaluation_strategy="epoch"
)

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


创建一个训练器对象，传入模型、训练参数、数据集等。

In [30]:
trainer = Trainer(
    model=model,  # the instantiated 🤗 Transformers model to be trained
    args=training_args,  # training arguments, defined above
    train_dataset=tokenized_train,  # training dataset
    eval_dataset=tokenized_val,  # evaluation dataset
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

开始训练。

In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss


对测试集进行预测，并打印预测结果。

In [ ]:
prediction_outputs = trainer.predict(tokenized_test)
test_pred = np.argmax(prediction_outputs[0], axis=-1).flatten()
print(test_pred)

将预测结果保存为CSV文件，并记录日志表示结果已保存。

In [ ]:
result_output = pd.DataFrame(data={"id": test["id"], "sentiment": test_pred})
result_output.to_csv("/kaggle/working/deberta_prompt.csv", index=False, quoting=3)
# result_output.to_csv("/kaggle/working/submission.csv", index=False, quoting=3)
logging.info('result saved!')